# Fast-Trips Tutorial 1: Run Fast-Trips for a Single Trip on Simple Network
## Install required software and run this notebook from a virtual environment

The easiest way to get this notebook to work locally is to [download Anaconda, for Python 2.7 ](https://www.continuum.io/downloads)

Then you will need to install dependencies, including:  

 - pandas  
 - transitfeed  
 - fast-trips  
 
This notebook also uses the following packages for visualization:

 - Folium [ for maps ]  
 - Bokeh  

In [ ]:
import os,datetime
import pandas as pd

# Specify Input Networks + Visualize

In [ ]:
BASE_DIR         = r"/Users/elizabeth/Documents/urbanlabs/MTC/SHRP2/tutorial/tta"
INPUT_NETWORKS   = os.path.join(BASE_DIR, r"input/network-simple")
INPUT_DEMAND     = os.path.join(BASE_DIR, r"input/demand-single")


## Validate GTFS Feed
Note that `transitfeed` does not expect all the extra files associated with GTFS-PLUS and it will be noisy about them.

In [ ]:
import transitfeed
loader        = transitfeed.Loader(INPUT_NETWORKS, memory_db=True)
schedule      = loader.Load()
schedule.Validate()

## Create Simple Map of Routes

In [ ]:
import folium
import tutorial_map

mymap = tutorial_map.make_map(schedule)
mymap

# Run Fast-Trips Example: Single Trip
Note that if you run this more than once, you will probably need to go to `kernel>>restart` menu so that fast-trips doesn't get confused by variables being held in memory.

In [ ]:
import fasttrips
from fasttrips import Run

In [ ]:
INPUT_WEIGHTS    = os.path.join(BASE_DIR, r"input/demand-single/pathweight_ft.txt")
RUN_CONFIG       = os.path.join(BASE_DIR, r"input/demand-single/config_ft.txt")
OUTPUT_FOLDER    = r"test_simplenet_noOverlap"
OUTPUT_DIR       = os.path.join(BASE_DIR, r"output")
ITERATIONS       = 1
PATHFINDING_TYPE = "stochastic"
OVERLAP          = "None"
DISPERSION       = 0.5

In [ ]:
Run.run_fasttrips(input_net_dir    = INPUT_NETWORKS,
                  input_demand_dir = INPUT_DEMAND,
                  run_config       = RUN_CONFIG,
                  input_weights    = INPUT_WEIGHTS,
                  output_dir       = OUTPUT_DIR,
                  output_folder    = OUTPUT_FOLDER,
                  pathfinding_type = PATHFINDING_TYPE,
                  iters            = ITERATIONS,
                  overlap_variable = OVERLAP,
                  dispersion       = DISPERSION)

# Examine Results

In [ ]:
from bokeh.charts import Bar, Histogram, TimeSeries, output_file, Line
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

## Vehicle/Route Level

In [ ]:
full_output_directory=os.path.join(OUTPUT_DIR,OUTPUT_FOLDER)
vehicles_df = pd.read_csv(os.path.join(full_output_directory,"veh_trips.csv"), 
                                       sep=",", 
                                       parse_dates=['arrival_time', 'departure_time'],
                                       date_parser=lambda x: datetime.datetime.strptime(x, '%H:%M:%S') )
#vehicles_df.head()

In [ ]:
max_iteration     = vehicles_df['iteration'].max()
max_pf_iteration  = vehicles_df['pathfinding_iteration'].max()

final_vehicles_df = vehicles_df[(vehicles_df['iteration'] == max_iteration) & (vehicles_df['pathfinding_iteration']==max_pf_iteration)]

In [ ]:
tooltips=[
    ('Trip ID', '@trip_id'),
]

all_lines  = Bar(final_vehicles_df, "route_id", values='boards', stack='trip_id',title="Boardings by Route",legend=False,
            xlabel="Route ID", ylabel="Boardings",tooltips=tooltips)

ridership = show(all_lines, notebook_handle=True)

## Person Level

In [ ]:
chosen_df = pd.read_csv(os.path.join(full_output_directory,"chosenpaths_links.csv"), 
                                       sep=",", 
                                       parse_dates=['new_A_time', 'new_B_time'],
                                       date_parser=lambda x: datetime.datetime.strptime(x, '%H:%M:%S') )



In [ ]:
pline = Line(chosen_df, x="linknum", y="new_A_time", color="linkmode",title="line", width=10)


show(pline, notebook_handle=True)


In [ ]:
import numpy as np

from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid,DatetimeAxis, LabelSet
from bokeh.models.glyphs import Segment
from bokeh.io import curdoc, show



palette = ["#d73838", "#dea23a", "#b1d149", "#5d85b5", "#9d56a4",
           "#f7f7f7", "#fddbc7", "#f4a582", "#d6604d", "#b2182b", "#67001f"]

mode = list(set(chosen_df["mode"]))
l    = len(mode)
mode_color = dict(zip(mode,palette[0:l]))


def assignColorByMode(mode):
    if mode in mode_color.keys():
        return mode_color[mode]
    return "#f0f0f0"
    
chosen_df["color"]= map(assignColorByMode, chosen_df["mode"])

def createAnnotation(mode,sim_cost,route_id,trip_id):
    return str(mode)+" "+str(sim_cost)+" "+str(route_id)+":"+str(trip_id)

chosen_df["annotation"]= map(createAnnotation, chosen_df["mode"],chosen_df["sim_cost"],chosen_df["route_id"],chosen_df["trip_id"])

source = ColumnDataSource(chosen_df)

In [ ]:
xdr = DataRange1d()
ydr = DataRange1d()

plot = Plot(
    title=None, x_range=xdr, y_range=ydr, plot_width=800, plot_height=300,
    h_symmetry=False, v_symmetry=False, min_border=0, toolbar_location=None)

glyph = Segment(y0="linknum", x0="new_A_time", y1="linknum", x1="new_B_time", line_color="color", line_width=20)
plot.add_glyph(source, glyph)

xaxis = DatetimeAxis()
plot.add_layout(xaxis, 'below')

yaxis = LinearAxis()
plot.add_layout(yaxis, 'left')

plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

mode_labels = LabelSet(x="new_A_time", y="linknum", text="mode", y_offset=-5,x_offset=10,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='left')
plot.add_layout(mode_labels)

curdoc().add_root(plot)

show(plot, notebook_handle=True)

## Path Choice

In [ ]:
PATHS  = r"pathset_paths.csv"

pathfile_df = pd.read_csv(os.path.join(full_output_directory,PATHS), sep=",")
pd.set_option('display.max_colwidth',160) #widen so you can see whole description
pathfile_df[["person_id","person_trip_id","pathnum","description","sim_cost","logsum","probability"]]

### Display links
Use the link file to display each component of the path and their respective costs.

In [ ]:
LINKS  = r"pathset_links.csv"
linkfile_df = pd.read_csv(os.path.join(full_output_directory,LINKS),
                                       sep=",", 
                                       parse_dates=['new_A_time', 'new_B_time'],
                                       date_parser=lambda x: datetime.datetime.strptime(x, '%H:%M:%S') )

## Add Fast-Trips path probability information to Fast-Trips link file
linkfile_df = pd.merge(left  = linkfile_df,
                       right = pathfile_df,
                       how   = "left",
                       left_on = ["person_id","person_trip_id","pathnum","chosen","missed_xfer"],
                       right_on = ["person_id","person_trip_id","pathnum","chosen","missed_xfer"])

#linkfile_df.head()
#linkfile_df.columns

In [ ]:

maxlinknum = linkfile_df["linknum"].max()
    
linkfile_df["color"]= map(assignColorByMode, linkfile_df["mode"])

def createAnnotation(mode,sim_cost,route_id,trip_id,probability):
    return str(mode)+" "+str(sim_cost)+" "+str(route_id)+":"+str(trip_id)

linkfile_df["annotation"]= map(createAnnotation, linkfile_df["mode"],linkfile_df["sim_cost_x"],linkfile_df["route_id"],linkfile_df["trip_id"],linkfile_df["probability"])

def yloc(pathnum,linknum):
    return (pathnum*(maxlinknum+1))+linknum

linkfile_df["yloc"]= map(yloc, linkfile_df["pathnum"],linkfile_df["linknum"])

linksource = ColumnDataSource(linkfile_df)
#linkfile_df

In [ ]:
xdr2 = DataRange1d()
ydr2 = DataRange1d()

choiceplot = Plot(
    title=None, x_range=xdr2, y_range=ydr2, plot_width=800, plot_height=300,
    h_symmetry=False, v_symmetry=False, min_border=0, toolbar_location=None)

glyph = Segment(y0="yloc", x0="new_A_time", y1="yloc", x1="new_B_time", line_color="color", line_width="sim_cost_x")
choiceplot.add_glyph(linksource, glyph)

choicexaxis = DatetimeAxis()
choiceplot.add_layout(choicexaxis, 'below')

choiceplot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))

choice_labels = LabelSet(x="new_A_time", y="yloc", text="annotation", y_offset=-5,x_offset=10,
                  text_font_size="8pt", text_color="#555555",
                  source=linksource, text_align='left')
choiceplot.add_layout(choice_labels)

curdoc().add_root(choiceplot)

##TODO add nodes
##TODO make annotations more visible; remove fields that aren't necessary for that link type
##TODO standardize colors by mode
##TODO add probability

show(choiceplot, notebook_handle=True)